In [21]:
import json, pickle
import pandas as pd
import importlib.util

file_path = "../kLLMmeans.py"  # Adjust path as needed

spec = importlib.util.spec_from_file_location("kLLMmeans", file_path)
kLLMmeans = importlib.util.module_from_spec(spec)
spec.loader.exec_module(kLLMmeans)

In [29]:
df = pd.read_csv('raw_stackexchange.csv')
print(df.shape)

groups_df = df[df.Year == 2023].groupby('Label').count().reset_index()
selected = list(groups_df[groups_df.Year>500].Label)
df = df[(df['Label'].isin(selected))&(df.Year>=2020)].reset_index(drop = True)
df.to_csv('clean_stackexchange.csv', index = False)

print(df.shape)

(499359, 4)
(205943, 4)


In [31]:
num_clusters = len(df.Label.unique()) # 35
df.groupby('Year').count().reset_index()

,Year,Text,CreationDate,Label
0,2020,69147,69147,69147
1,2021,54322,54322,54322
2,2022,43521,43521,43521
3,2023,38953,38953,38953


In [33]:
num_clusters

35

In [ ]:
for year in [2020,2021,2022,2023]:
    print('-' * 20)
    print(year)
    
    cur_df = df[df.Year == year].copy()
    documents = cur_df.Text
    
    for emb_type in ['openai']:
        data_dict = {}
        
        print(emb_type)
        embeddings = kLLMmeans.get_embeddings(list(documents), emb_type=emb_type, instructor_prompt = "")
    
        data_dict['data'] = cur_df
        data_dict['num_clusters'] = num_clusters
        data_dict['embeddings'] = embeddings
        
        with open("../processed_data/data_stackexchange_" + emb_type + "_" + str(year) + ".pkl", "wb") as f:
            pickle.dump(data_dict, f)

In [ ]:
for year in [2020,2021,2022,2023]:
    print('-' * 20)
    print(year)
    
    cur_df = df[df.Year == year].copy()
    documents = cur_df.Text
    
    for emb_type in ['openai']:
        sentences_dict = {}
       
        sents, index_map = NLPsummary.build_sentence_corpus(documents)
        print("sentence tokenizer ready")
        X, _ = NLPsummary.embed_sentences(sents, emb_type = emb_type)
        
        text_sentences = {i: [] for i in range(len(documents))}
        for i in range(len(sents)):
            cur_index = index_map[i][0]
            text_sentences[cur_index].append([sents[i],X[i]])
        sentences_dict[emb_type] = text_sentences
        
        with open("../processed_data/data_sentences_stackexchange_" + emb_type + "_" + str(year) + ".pkl", "wb") as f:
            pickle.dump(sentences_dict, f)